# TP2 Deep QLearning

Dans ce TP, vous devez implémenter un agent apprenant avec l'algorithme du Deep QLearning à faire atterir un vaisseau sur la lune, en utilisant l'environnement OpenAI Gym LunarLander-v2. 

<img src='img/lunarlander.png'  width=500px>

# 1. Consignes

> En plus de compléter ce notebook, vous devrez aussi:
- compléter les différents fichiers python associés. <span style="color:red"> Vous commenterez votre code</span> dans le notebook et dans les fichiers python.
- rédiger  un <span style="color:red">rapport en pdf</span>  qui sera ajouté à votre dépôt. 
> Ce rapport contiendra :

- le <span style="color:red">détail des différentes expérimentations réalisées dans les parties 6, 7 et 8 et une analyse des résultats</span>. Pour chaque apprentissage testé, préciser les différents hyper paramètres (architecture du réseau, coefficient(s) d'apprentissage, nombre d'épisodes, taille du buffer, décroissance d'epsilon, ... tout ce qui permet de reproduire l'expérience), mettre la courbe d'apprentissage obtenu, et analyser les résultats.

- <span style="color:red">l'analyse d'un article au choix</span> (cf. détails dans la partie 8 de ce notebook).  

> Le code doit être fonctionnel avec l'environnement virtuel du TP. Si d'autres packages que ceux présents dans l'environnement virtuel `nnPyTorch` sont nécessaires, vous devez ajouter à votre dépôt un fichier `environnement.yaml` qui est un export de votre environnement virtuel. Ce fichier est obtenu avec la commande suivante:  ```conda env export > environnement.yaml```



# 2. Import des packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

# import depuis un fichier python local 
from model import QNN 
from dqlagent import DQLAgent 
from greedyagent import GreedyAgent
import utils
from expereplay import ReplayBuffer
from dqlagent import DQLAgent
from dqlagenttarget import DQLAgentTarget

# 3. Gym

En apprentissage par renforcement, il y a deux concepts fondamentaux : l’agent et l’environnement. L’agent est l’entité apprenante qui observe l’environnement et agit sur celui-ci selon les actions disponibles. Son objectif est de maximiser la récompense cumulée qu’il recoit de l’environnement avec lequel il interagit. Chaque environnement a :
- Un espace d’action.
- Un espace d’état.
- Une fonction de récompense.

[OpenAI Gym](https://gym.openai.com) est une interface open source unifiée entre un agent et un environnement. Ainsi, il est possible de construire indépendamment un agent de l’environnement avec lequel il interagit, et inversement. Il suffit alors de satisfaire l’interface pour qu’un nouvel environnement soit compatible avec presque tous les agents précédemment codés pour d’autres environnements. 

Gym propose en particulier [un ensemble d'environnements](https://gym.openai.com/envs) pour des tâches d'apprentissage par renforcement. Une liste détaillée est disponible [ici](https://github.com/openai/gym/wiki/Table-of-environments). La plupart des environements ont leur code source disponible sur GitHub. De nouveaux environnements peuvent aussi être créés à condition qu'ils soient compatibles avec l'interface. 

Lorsque certains pré-traitements sont nécessaires sur les actions, observations ou récompenses, il est possible d’encapsuler l’environnement dans un wrapper, celui-ci se chargera du pré-traitement. 




Dans ce TP, nous allons implémenter un agent qui interagira avec l'environnement `LunarLander-v2`, et que l’on pourra ensuite tester sur d'autres environnements. Il existe plusieurs fonctions clé pour interagir avec un environnement.

>  <span style="color:green">Lisez la documentation de Gym</span>: [doc1](https://gym.openai.com/docs/#environments) et [doc2](https://github.com/openai/gym).




##  3.1 - Caractéristiques de l'environnement LunarLander-v2

> <span style="color:green">Compléter le code ci-dessous pour afficher **la taille des espaces d'états et d'actions** de l'environnement `LunarLander-v2`. Vous afficherez aussi un **échantillon pris au hasard dans chaque espace**. Préciser à quoi correspond **chaque élément de l'état et de l'action** (dans le notebook ou dans votre rapport).</span>

In [4]:
env = gym.make('LunarLander-v2')
env.seed(0)

#TODO


[0]

## 3.2 Affichage de l'environnement sur un épisode

> <span style="color:green">Exécuter une instance de l'environnement `LunarLander-v2` pendant un épisode avec des actions aléatoires. Afficher l'environnement à chaque pas pour visualiser le comportement du vaisseau et la somme des récompenses obtenues sur l'épisode. </span>

Un épisode débute lors du reset et se termine lorsqu’il recoit un signal de fin de la part de l’environnement.

In [5]:
#TODO


# 4. Agent $\epsilon$-glouton (sans apprentissage)

> <span style="color:green">Vous devez maintenant implémenter un **agent $\epsilon$-glouton** qui utilise un réseau de neurones pour représenter la Q fonction.</span>

- **Définir l'architecture du réseau de neurones** en complétant `model.py`. Ce réseau va approximer la Q-fonction comme dans le deep qlearning, selon une approximation *action vector* (pour l'instant, les poids du réseau ne seront pas mis à jour, le réseau est uniquement utilisé en prédiction mais les prédictions seront aléatoires).

- **Définir un agent $\epsilon-$glouton** en complétant `greedyagent.py`, qui utilisera la prédiction du réseau de neurones pour choisir ses actions selon une stratégie d’exploration $\epsilon$-greedy.

- **Tester cet agent** sur plusieurs épisodes dans `LunarLander-v2`. Vous utiliserez un décroissance de l'exploration, i.e. que $\epsilon$ va décroitre à chaque épisode, en démarrant à une valeur élevée (beaucoup d'exploration) et avec une borne minimum. Ainsi, au premier épisode, $\epsilon=\epsilon_{start}$, et à chaque épisode, $\epsilon=max(\epsilon_{end}, \epsilon_{decay}*\epsilon)$. Par exemplen sur 1000 épisodes, les valeurs peuvent être $\epsilon_{start} = 1.0$, $\epsilon_{end} = 0.01$ et $\epsilon_{decay} = 0.995$.

- **Proposer un tracé de la somme des récompenses obtenues par épisode** (et moyenne glissante) en utilisant le fichier `utils.py`, et une visualisation du dernier épisode.

- **Compléter votre rapport** (courbe, analyse du résultat, ...)


In [7]:
#TODO



# 5. Experience replay

L'algorithme du deep qlearning utilise l’experience replay:
- pendant la phase de **sample**, qui est une phase d'interaction avec l'environnement sans apprentissage, l'agent stocke en mémoire toutes les interactions (ou expériences) rencontrées. Une interaction est un tuple `(état,action,état_suivant,récompense,fin_episode)`.  Le buffer a une taille maximale (100 000 par exemple). Lorsqu’elle est dépassée, les nouvelles interactions remplacent les plus anciennes. 

- pendant la phase d'**apprentissage**, l’agent apprend à partir des interactions stockées dans son buffer. Il choisit aléatoirement un minibatch d’interactions dans son buffer (64 par exemple).

> <span style="color:green">Compléter la classe `ReplayBuffer` du fichier `expereplay.py` pour stockez les interactions et récupérer des mini-batchs. </span> 


# 6. Deep QLearning


Vous allez maintenant implémenter l'algorithme du **Deep QLearning** pour que votre agent apprenne en mettant à jour les poids de son réseau de neurone. 

- Après chaque interaction, le buffer sera remplit. 
- Une phase d'apprentissage sera réalisée toutes les `n` interactions (par ex. $n=4$), pendant laquelle un minibatch de données sera choisi dans le buffer pour réaliser la descente de gradient. Pour une interaction $(s,a,s',r)$, la valeur cible (erreur TD) sera calculée de la façon suivante:
    - $y = r(s,a) + \gamma max_b Q_\omega(s',b) $ si l'épisode continu
    - $y = r(s,a)  $ si l'épisode se termine

Remarque: vous n'utiliserez pas de réseau cible (*target network*) pour l'instant.


> <span style="color:green">Compléter la classe `DQLAgent` et utiliser cet agent sur plusieurs épisodes dans `LunarLander-v2`. Vous proposerez un tracé de la somme des récompenses obtenues par épisode (et moyenne glissante) en utilisant le fichier `utils.py`, et une visualisation du dernier épisode. **Compléter votre rapport**.

</span> 
 
Pour l'optimizer, SGD et [Adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) sont particulièrement adaptés. 

Voici aussi des liens vers différentes fonctions de PyTorch qui pourraient vous être utiles:
- [unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html)
- [gather](https://pytorch.org/docs/stable/generated/torch.gather.html)
 
 

In [ ]:
# TODO

# 7. Deep QLearning avec réseau cible

Il se peut que votre agent précédent apprenne des comportements intéressants mais qu’ils soient très instables. 

> <span style="color:green"> Compléter la classe `DQLAgentTarget` pour implémenter un agent apprenant avec le deep qlearning et un *target network* pour stabiliser l’apprentissage. Utiliser cet agent sur plusieurs épisodes dans `LunarLander-v2`. Vous proposerez un tracé de la somme des récompenses obtenues par épisode (et moyenne glissante) en utilisant le fichier `utils.py`, et une visualisation du dernier épisode. **Compléter votre rapport**.</span> 
 
Pour la mise à jour du réseau cible, vous pouvez:
- toutes les N étapes d'apprentissage (10000 par exemple), recopier entièrement le réseau de neurone original dans le duplicat
- mettre à jour petit à petit le duplicat à chaque étape d’apprentissage : $$\omega' = (1-\tau)\omega' +\alpha \omega$$ où $\omega'$ sont les poids du duplicat, $\omega$ les poids du réseau original, et $\tau$ le pas de mise à jour. Souvent $\tau = 0.001$.


Pour copier des poids d'un réseau de neurone vers un autre, la méthode `copy_` peut être appelée sur les paramètres:

`for param_duplicat, param_source in zip(model_duplicat.parameters(), model_source.parameters()):`
           
`param_duplicat.data.copy_(param_source.data)`






In [ ]:
# TODO

# 8. Extensions du Deep QLearning

Plusieurs extensions du Deep QLearning ont été proposés, dont les deux suivantes assez simples:

- [Double-DQN](https://arxiv.org/pdf/1509.06461.pdf)

- [Prioritized experience replay](https://arxiv.org/pdf/1511.05952.pdf)

De nombreux articles de blog présentent ces extensions, n'hésitez pas à les lire pour mieux comprendre ces articles (vous citerez alors ces références dans votre rapport).

> <span style="color:green"> Choisissez une des 2 extensions. Vous devez **lire l'article**, puis en faire **une analyse dans votre rapport**. Cet analyse comportera les éléments suivants: </span> 
- présenter le problème présent dans le deep QLearning classique que l'extension essaie de résoudre
- détailler la solution proposée (texte, équation, pseudo-code, ...)
- résumer les expérimentations réalisées dans l'article
- citer les références utilisées




# 9. Bonus

> Vous pouvez proposer **implémentation de l'extension** dans le fichier `dqlagentextension.py` et/ou tester votre algorithme dans d'autres [environnements](https://github.com/openai/gym/wiki/Table-of-environments).